# Pandas 重塑(stack)和轴向旋转(pivot)的实现
pandas中重新排列表格型数据的操作称为重塑（reshape）和轴向旋转（pivot）。可以将一个高维的数据按照想要的方式二维平铺展开。

In [2]:
import numpy as np
import pandas as pd

## 一、重塑
重塑操作方法有两个：

* stack:将数据的列索引旋转为行索引  
* unstack:将数据的行索引旋转为列索引  

In [6]:
data = pd.DataFrame(np.arange(9).reshape(3, 3), columns=pd.Index(list('abc'), name='title'), index=pd.Index(['aa', 'bb', 'cc'], name='list'))
data

title,a,b,c
list,,,
aa,0,1,2
bb,3,4,5
cc,6,7,8


In [8]:
df1 = data.stack()
df1

list  title
aa    a        0
      b        1
      c        2
bb    a        3
      b        4
      c        5
cc    a        6
      b        7
      c        8
dtype: int64

**默认stack将列索引转为行索引后，放在最内层**。在这里就得到了一个Series。而通过unstack可以逆操作，将行转为列：

In [9]:
df1.unstack()

title,a,b,c
list,,,
aa,0,1,2
bb,3,4,5
cc,6,7,8


**默认unstack也是转换最内层**，或者我们可以传入分层级别的编号或名称显式指定要转换的行。

分层级别编号从外到内依次0,1…，或者指定该行的name值也可以：

In [10]:
df1.unstack(0)

list,aa,bb,cc
title,,,
a,0,3,6
b,1,4,7
c,2,5,8


In [11]:
df1.unstack('list')

list,aa,bb,cc
title,,,
a,0,3,6
b,1,4,7
c,2,5,8


如果不是所有级别都能在各个分组中找到，unstack就会引入缺失值：

In [12]:
a1 = pd.Series(np.arange(4), index=list('abcd'))
a2 = pd.Series([4, 5, 6], index=list('cde'))
df2 = pd.concat([a1, a2], keys=['data1', 'data2'])
df2

data1  a    0
       b    1
       c    2
       d    3
data2  c    4
       d    5
       e    6
dtype: int64

In [13]:
# unstack行为列
df2.unstack()

,a,b,c,d,e
data1,0.0,1.0,2.0,3.0,NaN
data2,NaN,NaN,4.0,5.0,6.0


可以看出此时引入了缺失值，而使用stack默认会过滤缺失数据：

In [14]:
df2.unstack().stack()

data1  a    0.0
       b    1.0
       c    2.0
       d    3.0
data2  c    4.0
       d    5.0
       e    6.0
dtype: float64

或者使用dropna参数设定不过滤缺失值：

In [15]:
df2.unstack().stack(dropna=False)

data1  a    0.0
       b    1.0
       c    2.0
       d    3.0
       e    NaN
data2  a    NaN
       b    NaN
       c    4.0
       d    5.0
       e    6.0
dtype: float64

## 二、轴向旋转(pivot)

pivot(index,columns,values)：将index指定为行索引，columns是列索引，values则是DataFrame中的值

In [18]:
#假如有这样一个数据：
df3 = pd.DataFrame({'date':['2018-11-22','2018-11-22','2018-11-23','2018-11-23','2018-11-24'],                    
                    'item':['a','b','b','c','c'],
                    'values':[5,3,2,6,1]})
df3

,date,item,values
0,2018-11-22,a,5
1,2018-11-22,b,3
2,2018-11-23,b,2
3,2018-11-23,c,6
4,2018-11-24,c,1


事实上在数据库中，很多数据都是这样存储的。然而我们更希望将date当做索引，不同的item形成一列。因此就可以通过pivot()方法实现：

In [19]:
df3.pivot('date', 'item', 'values')

item,a,b,c
date,,,
2018-11-22,5.0,3.0,NaN
2018-11-23,NaN,2.0,6.0
2018-11-24,NaN,NaN,1.0


前两个参数分别当做行和列，最后的参数来填充数据列。如果忽略最后一个参数，就会得到一个层次化索引：

In [20]:
df3.pivot('date', 'item')

values          
item            a    b    c
date                       
2018-11-22    5.0  3.0  NaN
2018-11-23    NaN  2.0  6.0
2018-11-24    NaN  NaN  1.0

本质上pivot()方法是set_index和unstack结合使用的快捷方式：

In [21]:
df3.set_index(['date', 'item']).unstack('item')

values          
item            a    b    c
date                       
2018-11-22    5.0  3.0  NaN
2018-11-23    NaN  2.0  6.0
2018-11-24    NaN  NaN  1.0

In [22]:
df3.set_index(['date', 'item'])

values
date       item        
2018-11-22 a          5
           b          3
2018-11-23 b          2
           c          6
2018-11-24 c          1